In [ ]:
!unzip -qq '/content/drive/My Drive/Cherry.zip'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
train_transforms = transforms.Compose([
                            transforms.Resize(224),
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(224, padding = 10),
                            transforms.ToTensor()
                                      ])
train_data = datasets.ImageFolder(root = '/content/Cherry',transform=train_transforms)
RATIO = 0.8
n_train_examples = int(len(train_data) * RATIO)
n_orginalvalid_examples = len(train_data) - n_train_examples
n_valid_examples = int((n_orginalvalid_examples)*0.5)
n_test_examples = n_orginalvalid_examples - n_valid_examples
print(n_train_examples)
print(n_valid_examples)
print(n_test_examples)

train_data, valid_data, test_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples, n_test_examples])

1524
191
191


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import copy
import random
import time
import torchvision.models as models


In [ ]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features,out_features=2)

for param in model.parameters():
  param.requires_grad=True
    


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


In [ ]:
# import copy
# import random
# import time
# import torchvision.models as models
# model = models.resnet18(pretrained=True).to(device)

# for param in model.parameters():
#   param.requires_grad=True
    
# model.fc = nn.Sequential(
#                nn.Linear(model.fc.in_features, 496),
#                nn.ReLU(inplace=True),
#                nn.Linear(496, 4)).to(device)

In [ ]:
model = model.to(device)

In [ ]:
BATCH_SIZE = 16
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,226,434 trainable parameters


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),weight_decay=0.01, lr=0.00001)
criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 200

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.280 | Train Acc: 92.90%
	 Val. Loss: 0.066 |  Val. Acc: 100.00%
Epoch: 02 | Epoch Time: 0m 9s
	Train Loss: 0.063 | Train Acc: 99.54%
	 Val. Loss: 0.021 |  Val. Acc: 100.00%
Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.032 | Train Acc: 99.54%
	 Val. Loss: 0.009 |  Val. Acc: 100.00%
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.034 | Train Acc: 99.02%
	 Val. Loss: 0.009 |  Val. Acc: 100.00%
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.021 | Train Acc: 99.61%
	 Val. Loss: 0.005 |  Val. Acc: 100.00%
Epoch: 06 | Epoch Time: 0m 9s
	Train Loss: 0.012 | Train Acc: 99.93%
	 Val. Loss: 0.003 |  Val. Acc: 100.00%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 0.018 | Train Acc: 99.67%
	 Val. Loss: 0.003 |  Val. Acc: 100.00%
Epoch: 08 | Epoch Time: 0m 9s
	Train Loss: 0.011 | Train Acc: 99.87%
	 Val. Loss: 0.002 |  Val. Acc: 100.00%
Epoch: 09 | Epoch Time: 0m 9s
	Train Loss: 0.009 | Train Acc: 99.93%
	 Val. Loss: 0.002 |  Val. Acc: 100.00%
Epoch: 10 | Epoch 

KeyboardInterrupt: ignored